# 4.5 - PySpark

$$$$

![pyspark](images/pyspark.jpg)

$$$$

Apache Spark es un framework de computación en clúster open-source. Fue desarrollada originariamente en la Universidad de California, en el AMPLab de Berkeley. El código base del proyecto Spark fue donado más tarde a la Apache Software Foundation que se encarga de su mantenimiento desde entonces. Spark proporciona una interfaz para la programación de clusters completos con Paralelismo de Datos implícito y tolerancia a fallos.

Apache Spark se puede considerar un sistema de computación en clúster de propósito general y orientado a la velocidad. Proporciona APIs en Java, Scala, Python y R. También proporciona un motor optimizado que soporta la ejecución de grafos en general. También soporta un conjunto extenso y rico de herramientas de alto nivel entre las que se incluyen Spark SQL (para el procesamiento de datos estructurados basada en SQL), MLlib para implementar machine learning, GraphX para el procesamiento de grafos y Spark Streaming.

In [ ]:
%pip install findspark
%pip install pyspark

In [ ]:
import warnings
warnings.simplefilter('ignore')

import findspark
findspark.init() 

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName('Nombre').getOrCreate()  # inicia la sesion de spark

path='../data/student-por.csv'

In [ ]:
data=spark.read.csv(path, header=True, inferSchema=True, sep=';')

data.show(5)

In [ ]:
display(data.show(5))

In [ ]:
drop_cols=['school', 'sex', 'age', 'Mjob', 'Fjob', 'reason', 'guardian']

data=data.select([c for c in data.columns if c not in drop_cols])

data

In [ ]:
non_numeric_columns=[item[0] for item in data.dtypes if item[1].startswith('string')]

non_numeric_columns

In [ ]:
struct_data=data.select('*')

struct_data

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.sql.types import IntegerType


indexers=StringIndexer(inputCols=non_numeric_columns, 
                       outputCols=[c+'_' for c in non_numeric_columns],
                       stringOrderType='alphabetAsc')

struct_data=indexers.fit(struct_data).transform(struct_data)

struct_data=struct_data.select([c for c in struct_data.columns if c not in non_numeric_columns])

for c in struct_data.columns:
    struct_data=struct_data.withColumn(c, struct_data[c].cast(IntegerType()))

    
struct_data.toPandas()

### Ejemplo: aproximando $\pi$

Utilizaremos el Método de MonteCarlo para aproximar el número $\pi$. El método Monte Carlo es un método en el que por medio de la estadística y la probabilidad podemos determinar valores o soluciones de ecuaciones que calculados con exactitud son muy complejas, pero que mediante este método resulta sencillo calcular una aproximación al resultado que buscamos.

$$$$

![pi](images/pi.png)

$$$$

Lo primero construir el entorno de trabajo. Este sería:

+ Construiremos un cuadrado de lado 1.
+ Construimos un círculo inscrito en el cuadrado, que tiene de centro, el centro del cuadrado y de radio 1. Su área será $\pi$.
+ Generaremos puntos al azar dentro del cuadrado. Para entenderlo mejor es como lanzar dardos sobre una diana con los ojos vendados, de tal forma que siempre acertamos dentro de los límites de ese cuadrado. 

Aplicamos ahora el Método MonteCarlo:
+ Contaremos el total de puntos generados.
+ Contaremos el total de puntos que cayeron dentro del círculo.
+ Realizaremos el siguiente razonamiento:

$$A0 =  Área_{cuadrado} = N_{puntos} $$
$$$$
$$A1 = Área_{círculo} = \pi · r^{2}$$

Ahora:

$$\frac{\pi · r^{2}}{N_{puntos}} = \frac{Área_{círculo}}{Área_{cuadrado}}$$

Resumiremos en un cuadrante, y los que nos queda es que:

$$\pi=4·Área_{cuadrante}$$

El valor de $\pi$ es 4 veces la probabilidad de que el punto caiga en la zona roja.

In [ ]:
import numpy as np
from pyspark import SparkContext

In [ ]:
# puntos aleatorios dentor del círculo

def dentro(punto):
    x, y = np.random.random(), np.random.random()
    return x*x + y*y < 1

In [ ]:
def estimar_pi(n_total):
    print('Proceso normal...')

    puntos=list(filter(dentro, list(range(n_total)))) 
    
    cuenta=len(puntos)
  
    return 4. * cuenta/n_total

In [ ]:
%%time
display(estimar_pi(5000))
display(estimar_pi(50000))
display(estimar_pi(5000000))
        
display('Valor real pi: ' ,np.pi)

**con spark**

In [ ]:
sesion=SparkContext.getOrCreate()

In [ ]:
def estimar_pi_paralelo(n_total):
    print('Proceso con Spark..')

    cuenta=sesion.parallelize(range(0, n_total)).filter(dentro).count()

    return 4. * cuenta/n_total

In [ ]:
%%time
display(estimar_pi_paralelo(5000))
display(estimar_pi_paralelo(50000))
display(estimar_pi_paralelo(5000000))
        
display('Valor real pi: ' ,np.pi)